# Prerequisites

- Fields
- GCM-AES

# Theory

- https://meowmeowxw.gitlab.io/ctf/utctf-2020-crypto/

**Task**: 
- Given 2 signed pairs that are computed using the same nonce compute a valid tag for a 3rd ciphertext

### GHASH reminder


GHASH works in the field $GF(2^{128}) \bmod  X^{128} + X^7 + X^2 + X + 1$
- The field has coefficients in $\{0,1\}$
- Strings of 128 bits can be converted into polynomials -> the bits are the coefficients

For 
- a key $k∈GF(2^{128})$ 
- an input vector $z = (z_0, z_1, ... z_{v-1}) ∈(GF(2^{128}))^v$

$$GHASH(k, z) = z_0k^v + z_1k^{v-1} + z_{v-1}k \in GF(2^{128})$$

for our case $z = d'|| c' || len(d) || len(c)$

Consider the following variables
- $c$ = ciphertext
- $d$ = additional data
- $m$ = plaintext
- $t$ = tag
- $l$ = the lenghts of $d$ and $c$
- $s$ = the secret encryption of the $ctr$

let $g(X)$ be our GHASH function 
- $g(X) =d_1X^{m+n+1}+\dots+d_mX^{n+2}+c_1X^{n+1}+\dots+C^nX_2+lX+s$

To simplify we'll consider 2 messages that have 1 block of data and 1 block of ciphertext
- $g_1(X) = d_1X^3 + c_1X^2 + l_1X + s$
- $g_2(X) = d_2X^3 + c_2X^2 + l_2X + s$
- Notice that $s$ is the same since they use the same nonce

Inserting the key $H$
- $g_1(H) = t_1$
- $g_2(H) = t_2$


**Note**
- An attacker knows everything but $H, s$

**Let's add the polynomials**

- $g_1(H) + g_2(H) = t_1 + t_2 = (d_1+d_2)H^3 + (c_1+c_2)H^2 + (l_1+l_2)H + \underbrace{s + s}_0$
- $0 = (d_1+d_2)H^3 + (c_1+c_2)H^2 + (l_1+l_2)H + t_1 + t_2 = p(H)$ for some polynomial $p(X)$
- We get the roots of this equation / $p(X)$ => $H$


Having found $H$ we can compute $s$
- $s = g_1(H) + t_1$

# Code (Sage)

In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.number import long_to_bytes, bytes_to_long
from Crypto.Util import Counter

In [2]:
# Helper functions
def bytes_to_poly(byte_string, poly_generator):
    poly = 0
    byte_string_padded = byte_string + b'\x00' * (-len(byte_string) % 16) # front pad with 0
    #print(byte_string_padded)
    bit_string = bin(bytes_to_long(byte_string_padded))[2:].zfill(128)
    #print(bit_string)
    
    for i, bit in enumerate(bit_string):
        poly+= (poly_generator**i) * int(bit)
    return poly

def poly_to_bytes(poly):
    return long_to_bytes(int(bin(poly.integer_representation())[2:].zfill(128)[::-1], 2))

def get_length(c, d):
    len_c = long_to_bytes(len(c) * 8)
    len_c_ = b'\x00' * (-len(len_c) % 8)  + len_c # front pad with 0
    len_d = long_to_bytes(len(d) * 8)
    len_d_ = b'\x00' * (-len(len_d) % 8)  + len_d # frontpad with 0
    lens = len_d_ + len_c_
    return lens


def _to_gf2e(n, F):
    return F([(n >> i) & 1 for i in range(127, -1, -1)])

In [17]:
#msg1 = b'msg1111111'
#msg2 = b'msg24sad'
#msg3 = b'msg3v4444'
msg1 = get_random_bytes(15)
msg2 = get_random_bytes(14)
msg3 = get_random_bytes(13)

#data1 = b'data1132'
#data2 = b'data23213'
#data3 = b'data333'

data1 = get_random_bytes(12)
data2 = get_random_bytes(11)
data3 = get_random_bytes(10)

nonce = b'12Byte_Nonce'
key = get_random_bytes(16)
key = b'random16-Bytekey'

In [18]:
cipher = AES.new(key=key, mode=AES.MODE_GCM, nonce=nonce)
cipher.update(data1)
c1, t1 = cipher.encrypt_and_digest(msg1)
l1 = get_length(c1, data1)

cipher = AES.new(key=key, mode=AES.MODE_GCM, nonce=nonce)
cipher.update(data2)
c2, t2 = cipher.encrypt_and_digest(msg2)
l2 = get_length(c2, data2)

cipher = AES.new(key=key, mode=AES.MODE_GCM, nonce=nonce)
cipher.update(data3)
c3, _ = cipher.encrypt_and_digest(msg3)
l3 = get_length(c3, data3)



In [19]:
l2, l1

(b'\x00\x00\x00\x00\x00\x00\x00X\x00\x00\x00\x00\x00\x00\x00p',
 b'\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00x')

In [20]:
# The fields we work in
PF.<x> = GF(2)[]
F.<a>= GF(2**128, modulus = x**128 + x**7 + x**2 + x + 1) #d, c, t, l are here
PR.<X> = PolynomialRing(F) #g(X), p(X) is here

In [21]:
bytes_to_poly(c1, a)

a^119 + a^115 + a^113 + a^111 + a^110 + a^109 + a^108 + a^103 + a^99 + a^97 + a^95 + a^91 + a^90 + a^89 + a^88 + a^87 + a^81 + a^79 + a^78 + a^77 + a^75 + a^73 + a^71 + a^70 + a^67 + a^64 + a^60 + a^58 + a^57 + a^56 + a^55 + a^53 + a^52 + a^51 + a^49 + a^48 + a^46 + a^44 + a^42 + a^41 + a^35 + a^33 + a^32 + a^30 + a^29 + a^27 + a^26 + a^24 + a^23 + a^21 + a^19 + a^17 + a^16 + a^14 + a^13 + a^11 + a^10 + a^9 + a^7 + a^6 + a^5 + a^2 + 1

In [22]:
_to_gf2e(bytes_to_long(c1), F)

a^127 + a^123 + a^121 + a^119 + a^118 + a^117 + a^116 + a^111 + a^107 + a^105 + a^103 + a^99 + a^98 + a^97 + a^96 + a^95 + a^89 + a^87 + a^86 + a^85 + a^83 + a^81 + a^79 + a^78 + a^75 + a^72 + a^68 + a^66 + a^65 + a^64 + a^63 + a^61 + a^60 + a^59 + a^57 + a^56 + a^54 + a^52 + a^50 + a^49 + a^43 + a^41 + a^40 + a^38 + a^37 + a^35 + a^34 + a^32 + a^31 + a^29 + a^27 + a^25 + a^24 + a^22 + a^21 + a^19 + a^18 + a^17 + a^15 + a^14 + a^13 + a^10 + a^8

In [23]:
poly_to_bytes(bytes_to_poly(msg1, a))

b'{\x08\n5\xdf\xbc\xbb\x86]@\x97\x80ev\xd7\x00'

In [24]:
#Transform the bytes into polynomials
C1 = bytes_to_poly(c1, a)
C2 = bytes_to_poly(c2, a)
C3 = bytes_to_poly(c3, a)

T1 = bytes_to_poly(t1, a)
T2 = bytes_to_poly(t2, a)

D1 = bytes_to_poly(data1, a)
D2 = bytes_to_poly(data2, a)
D3 = bytes_to_poly(data3, a)

L1 = bytes_to_poly(l1, a)
L2 = bytes_to_poly(l2, a)
L3 = bytes_to_poly(l3, a)


In [25]:
#Add the polynomials and tags
p = (D1 + D2)*X**3 + (C1 + C2)*X**2 + (L1 + L2)*X + (T1 + T2)
print(p)

(a^94 + a^93 + a^91 + a^90 + a^87 + a^84 + a^80 + a^73 + a^71 + a^70 + a^69 + a^66 + a^64 + a^57 + a^56 + a^54 + a^52 + a^45 + a^44 + a^41 + a^37 + a^32 + a^31 + a^29 + a^27 + a^26 + a^25 + a^23 + a^21 + a^20 + a^19 + a^14 + a^12 + a^11 + a^10 + a^9 + a^6 + a^3 + a^2 + 1)*X^3 + (a^119 + a^115 + a^113 + a^111 + a^110 + a^107 + a^106 + a^105 + a^102 + a^101 + a^100 + a^97 + a^96 + a^91 + a^90 + a^89 + a^88 + a^87 + a^86 + a^85 + a^83 + a^82 + a^80 + a^79 + a^77 + a^76 + a^75 + a^71 + a^70 + a^69 + a^64 + a^63 + a^62 + a^61 + a^60 + a^58 + a^57 + a^54 + a^53 + a^52 + a^49 + a^47 + a^45 + a^44 + a^43 + a^41 + a^40 + a^30 + a^29 + a^28 + a^27 + a^26 + a^25 + a^24 + a^23 + a^21 + a^19 + a^17 + a^15 + a^14 + a^12 + a^11 + a^10 + a^8 + a^7 + a^6 + a^5 + a^2)*X^2 + (a^124 + a^60 + a^59 + a^58)*X + a^127 + a^124 + a^123 + a^121 + a^120 + a^115 + a^112 + a^111 + a^110 + a^109 + a^108 + a^106 + a^105 + a^104 + a^103 + a^101 + a^100 + a^97 + a^96 + a^95 + a^94 + a^91 + a^90 + a^89 + a^88 + a^87 + a

In [26]:
poly_to_bytes(C1)

b'\xa7v\xd5\xb6\xd0j\xdd\xe8\x93WA\xf1Q\x0fQ\x00'

In [27]:
# Find the GHASH key
Hs = p.roots()
print("The roots:", Hs)
print()
if Hs:
    H = Hs[0][0]
    print("Take a root H = ", H)

The roots: [(a^126 + a^123 + a^120 + a^116 + a^114 + a^113 + a^109 + a^108 + a^107 + a^105 + a^103 + a^102 + a^100 + a^99 + a^97 + a^96 + a^94 + a^87 + a^86 + a^84 + a^83 + a^82 + a^81 + a^79 + a^78 + a^77 + a^75 + a^74 + a^72 + a^70 + a^69 + a^66 + a^65 + a^62 + a^60 + a^58 + a^57 + a^56 + a^55 + a^54 + a^51 + a^50 + a^49 + a^48 + a^47 + a^46 + a^44 + a^43 + a^42 + a^40 + a^37 + a^36 + a^34 + a^33 + a^30 + a^28 + a^27 + a^25 + a^24 + a^23 + a^21 + a^18 + a^17 + a^16 + a^14 + a^10 + a^9 + a^8 + a^7 + a^3 + a^2, 1), (a^126 + a^124 + a^122 + a^119 + a^118 + a^116 + a^114 + a^113 + a^112 + a^110 + a^109 + a^106 + a^104 + a^103 + a^100 + a^99 + a^97 + a^94 + a^92 + a^90 + a^89 + a^88 + a^87 + a^86 + a^80 + a^78 + a^76 + a^74 + a^73 + a^72 + a^69 + a^67 + a^66 + a^65 + a^63 + a^62 + a^61 + a^60 + a^58 + a^56 + a^55 + a^52 + a^49 + a^48 + a^47 + a^43 + a^41 + a^40 + a^36 + a^35 + a^34 + a^33 + a^26 + a^24 + a^21 + a^20 + a^16 + a^15 + a^13 + a^11 + a^9 + a^8 + a^4 + a^3 + a + 1, 1), (a^126 +

In [28]:
# Find the Nonce encryption
S = D1*H**3 + C1*H**2 + L1*H + T1
print('S = ', S, S == D2*H**3 + C2*H**2 + L2*H + T2)

S =  a^126 + a^124 + a^123 + a^122 + a^121 + a^119 + a^113 + a^112 + a^109 + a^108 + a^105 + a^99 + a^96 + a^95 + a^93 + a^92 + a^90 + a^88 + a^86 + a^85 + a^84 + a^80 + a^79 + a^78 + a^75 + a^73 + a^72 + a^71 + a^70 + a^61 + a^60 + a^59 + a^58 + a^56 + a^54 + a^53 + a^50 + a^44 + a^43 + a^41 + a^39 + a^38 + a^37 + a^36 + a^35 + a^33 + a^31 + a^29 + a^28 + a^27 + a^26 + a^22 + a^21 + a^15 + a^14 + a^13 + a^12 + a^10 + a^8 + a^7 + a^6 + a^5 + a^4 + a^3 True


In [29]:
# Compute a forgery
T3 = D3*H**3 + C3*H**2 + L3*H + S
t3_forged = poly_to_bytes(T3)
t3_forged, len(t3_forged)

(b'A\xc4O\xe0Ls\xedY\xeb\xa7\xd2\xbei\xaf33', 16)

In [30]:
cipher = AES.new(key=key, mode=AES.MODE_GCM, nonce=nonce) #Nonce and key are unknown
cipher.update(data3)
#cipher.decrypt(c3)
msg3_decr = cipher.decrypt_and_verify(c3, t3_forged)
msg3_decr == msg3

True

# Resources

- https://toadstyle.org/cryptopals/63.txt
- https://csrc.nist.gov/csrc/media/projects/block-cipher-techniques/documents/bcm/comments/800-38-series-drafts/gcm/joux_comments.pdf
